<a href="https://colab.research.google.com/github/aishwarya-kumar/skillrec_for_gigworkers/blob/main/LLM_pipeline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !pip install langchain
# !pip install langchain_community
# !pip install transformers torch

In [1]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [3]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [6]:
# hf_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if device == 'cuda' else -1)
hf_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=0 if device == "cuda" else -1,
    max_length=200,
    max_new_tokens=150
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

<ipython-input-6-7565a7aea5b4>:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [7]:
market_trends_prompt = PromptTemplate(
    input_variables=["market_trends"],
    template="""The following are the latest market trends: {market_trends}
Explain to the user the top 3 in-demand job roles and the top 5 skills needed for each role in a clear and concise manner.""")

market_trends_chain = LLMChain(llm=llm, prompt=market_trends_prompt, output_key="market_explanation")

<ipython-input-7-1ed845e96a31>:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  market_trends_chain = LLMChain(llm=llm, prompt=market_trends_prompt, output_key="market_explanation")


In [8]:
skill_mapping_prompt = PromptTemplate(
    input_variables=["user_skills", "market_trends"],
    template="""The user has the following skills: {user_skills}.
Based on the market trends provided: {market_trends},
1. Identify the top in-demand skills the user already has.
2. Map the user's skills to transferable skills for the top job roles.""")

skill_mapping_chain = LLMChain(llm=llm, prompt=skill_mapping_prompt, output_key="skill_mapping")

In [9]:
income_comparison_prompt = PromptTemplate(
    input_variables=["current_income", "market_income"],
    template="""The user's current income is {current_income} USD per hour.
The market average incomes for the top job roles are as follows: {market_income}
Identify the roles with higher income potential than the user's current role.""")

income_comparison_chain = LLMChain(llm=llm, prompt=income_comparison_prompt, output_key="income_comparison")

In [10]:
career_recommendation_prompt = PromptTemplate(
    input_variables=["skill_mapping", "income_comparison"],
    template="""Based on the skill mapping results:
{skill_mapping}
And the income comparison results:
{income_comparison}
Recommend the top career role the user should switch to. Identify the skills needed for this role.""")

career_recommendation_chain = LLMChain(llm=llm, prompt=career_recommendation_prompt, output_key="career_recommendation")

In [11]:
upskilling_prompt = PromptTemplate(
    input_variables=["career_recommendation"],
    template="""The user should switch to the following career role: {career_recommendation}

Provide a detailed guide on how the user can obtain the required skills for this career switch.""")

upskilling_chain = LLMChain(llm=llm, prompt=upskilling_prompt, output_key="upskilling_directions")

In [12]:
pipeline = SequentialChain(
    chains=[market_trends_chain, skill_mapping_chain, income_comparison_chain, career_recommendation_chain, upskilling_chain],
    input_variables=["market_trends", "user_skills", "current_income", "market_income"],
    output_variables=["market_explanation", "skill_mapping", "income_comparison", "career_recommendation", "upskilling_directions"]
)

In [13]:
user_input = {
    "market_trends": """1. Data Scientist: Python, SQL, Machine Learning, Data Visualization, Cloud Computing
2. Software Engineer: Java, C++, Git, Problem-Solving, Systems Design
3. UX Designer: Wireframing, Prototyping, User Research, Figma, Graphic Design""",
    "user_skills": "Python, SQL, Data Visualization, Marketing Analytics",
    "current_income": "40",
    "market_income": """Data Scientist: 60 USD/hour
Software Engineer: 50 USD/hour
UX Designer: 45 USD/hour"""
}

In [14]:
results = pipeline.invoke(user_input)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/te

In [15]:
print("Market Trends Explanation:\n", results["market_explanation"])
print("Skill Mapping:\n", results["skill_mapping"])
print("Income Comparison:\n", results["income_comparison"])
print("Career Recommendation:\n", results["career_recommendation"])
print("Upskilling Directions:\n", results["upskilling_directions"])

Market Trends Explanation:
 The following are the latest market trends: 1. Data Scientist: Python, SQL, Machine Learning, Data Visualization, Cloud Computing
2. Software Engineer: Java, C++, Git, Problem-Solving, Systems Design
3. UX Designer: Wireframing, Prototyping, User Research, Figma, Graphic Design
Explain to the user the top 3 in-demand job roles and the top 5 skills needed for each role in a clear and concise manner. Here are the explanations:
**Data Scientist:**
Data scientists are the go-to professionals for organizations looking to make data-driven decisions. They are in high demand, and their skills are in high demand. Here are the top 3 in-demand job roles and the top 5 skills needed for each role:
* **Job Role 1: Business Intelligence Analyst**
	+ Job Description: Analyze and interpret complex data to identify trends and patterns, and present findings to stakeholders.
	+ Required Skills:
		1. Data Analysis and Visualization
		2. Statistical Modeling
		3. Machine Learning